In [1]:
# import reddit api wrapper
import praw
import pandas as pd
import numpy as np
import datetime
import sqlite3

from textblob import TextBlob

# Path hack
import os
# change directory from the current Analysis folder to the top level folder for easier navigation
os.chdir('../')
# confirm we're at /RedditTextAnalysis
print(os.getcwd())

# note that the %load_ext autoreload line only needs to be be run once
%load_ext autoreload

/Users/prcork/MiscDataProjects/collaboration/RedditTextAnalysis


In [2]:
# by including this autoreload command, we only need to re-import Spatial_Joins if we make/save changes to the original py file
%autoreload
from Functions.nfl_gamethreads import nfl_gamethreads

In [3]:
# access password and client secret id via local files
with open('pw.txt', 'r') as file1:
    pw = file1.read()

with open('client_secret.txt', 'r') as file2:
    cs = file2.read()

# create a praw Reddit instance with app credentials and secret info passed through
reddit = praw.Reddit(
    client_id="XbesrQBvKymjgLdgg_D6lA",
    client_secret=cs,
    user_agent="NFLTextAnalysis/0.0.1",
    username="ta_api",
    password=pw
)

In [4]:
# read in the gamethreads ids from sql
gamethreads = pd.read_csv('Data/gamethreads_2021.csv')

In [5]:
with sqlite3.connect("nfl_gamethreads.db") as conn:

    # pull in all submission ids that have previously been crawled
    df = pd.read_sql_query("SELECT * FROM comments", conn)
    crawled_ids = df['submission_id'].unique().tolist()

    all_ids = gamethreads['id'].unique().tolist()

    # yields the elements in all_ids not in crawled_ids
    remaining_ids = np.setdiff1d(all_ids, crawled_ids)

    for each_id in remaining_ids:

        comment_list = nfl_gamethreads.get_comments(reddit, each_id)

        comment_df = pd.DataFrame(comment_list, columns=['comment_id', 'submission_id', 'author', 'body', 'upvotes', 'utc_time', 'author_flair'])
        
        comment_df.to_sql(name='comments', con=conn, if_exists='append', index=False)



Submission ID: qaapr1
Comments: 23087
Comments stored.
Submission ID: qaz2fd


TooManyRequests: received 429 HTTP response